# TO DO:

- Make generic functions for tasks for modularity
- CRON jobs for all timeframes
- One function to update all csvs
- Plug n play for indicators
- Add limit to queries
- Include Active/Inactive column in CSV for active/inactive coins
- Update_CSV_to_Latest should contain active coins from Exchanges and From CSV. Check if a (coin,exchange) tuple is active(Check if it is present on exchange using ccxt library) . If it is active   get latest data for it if already present in CSV,if not in CSV get all data. If coin is not active on exchange , we will put a Active/Inactive status in CSV accordingly. All functions will have   to be modified to run code only for active coin-exchange combinations.
- Date Format will be '%d-%m-%Y %H:%M:%S' . This is giving me a lot of problems especially while reading data. When I don't put :%S it tells me dataframe has second and sometimes when :%S is         there, it tells me no second value in dataframe.
- Analytics Value Accuracy. Some parameter in Jupyter.
- Have to fetch Coins based on Parameters. Example - Fetch all active coins-exchange combinations where RSI>0 and RSI<=30. Fetch all active coins-exchange combinations where closing price is         between LOWERBAND and MIDDLEBAND. Get me intersection(common coins) of these 2 list. Now the coin from the intersection list which probably has the lowest volume can increase in price faster       then the others(Little increase in Volume will result in Big increase in Price)
- For each active coin-exchange combination I want to check the change in Value of different Technical Indicators of 2 consecutive periods in time. Example - I want to know if for a particular       coin RSI=a on period x and RSI>a on period x+1. I want to know whenever MACD and MACD_SIGNAL cross each other(On period x MACD=a and MACD_SIGNAL=b where a<=b and on period x+1 MACD=a and           MACD_SIGNAL=b where a>b. MACD_HISTOGRAM same like RSI want to know when it is 'a' on period x and 'a++' on period x+1).
- Convert 1D timeframe to 3D/1Week/etc. Convert 1H timeframe to 4H/6H/etc.
- Batch Processing of downloading new coins data


In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} yaml

In [ ]:
#import sys
#!{sys.executable} -m pip install pyyaml

In [ ]:
print(yaml.__file__)


In [ ]:
import os
import time
#import talib
import logging
import requests
import datetime
import importlib
import dateutil.parser
import ccxt
import numpy as np
import pandas as pd
from importlib import reload
from collections import defaultdict
import sys
import inspect
import pyti
import pprint
from apscheduler.schedulers.background import BackgroundScheduler
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import yaml

In [ ]:
coinmetrics_wrapper = 'coinmetrics_wrapper.py'
coinmetrics_wrapper

In [ ]:
import coinmetrics_wrapper as coinmetrics
reload(coinmetrics)

In [ ]:
importlib.reload(logging)
LOGGING_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=LOGGING_FORMAT, level=logging.
                    INFO)
#sqlalchemy.__version__
pd.__version__

In [ ]:
# FIELDS
PRICE = 'PRICE'
HIGH = 'HIGH24HOUR'
LOW = 'LOW24HOUR'
VOLUME = 'VOLUME24HOUR'
CHANGE = 'CHANGE24HOUR'
CHANGE_PERCENT = 'CHANGEPCT24HOUR'
MARKETCAP = 'MKTCAP'
NPERIODS = 100
TIMEFRAME = 'Day'
datetimeStringformat_to_csv = "%d-%m-%Y %H:%M"
one_day_in_milliseconds = 86400000
data_directory = '/Users/ankitaggarwal/Applications/Cryptocurrency Trading System/crypto-analysis/data/'

In [ ]:
# Defaults
# Get All the exchange objects
bittrex_exchange = ccxt.bittrex()
binance_exchange = ccxt.binance()
coinbasePro = ccxt.coinbasepro()
"""
kucoin_exchange = ccxt.kucoin()
huobiPro_exchange = ccxt.huobipro()
bitmex_exchange = ccxt.bitmex()
"""


list_of_exchanges = [coinbasePro,binance_exchange]
function_period_mapping = {
    'assets': coinmetrics.get_coin_list,
    'exchanges': coinmetrics.get_exchanges_list,
    'markets': coinmetrics.get_markets_list,
    'metrics' : coinmetrics.get_metrics_list,
    'metric_info' : coinmetrics.get_metrics_info_list
}

In [ ]:
# Limit of Results each exchange Sends over HTTPS Request
limit_of_trades_to_be_downloaded_from_exchange = {binance_exchange :500,coinbasePro:100} # got this from online API Documentation

# Print All the Keys of the Exchange Variable

In [ ]:
pprint.pprint(vars(coinbasePro))

In [ ]:
pprint.pprint(vars(binance_exchange))

In [ ]:
binance_exchange.milliseconds()

In [ ]:
binance_exchange.to_datetime()


# For All Exchanges, Get it's Market

In [ ]:
# For every exchange, fetch it's markets. Then depending on the JSON returned, prepare a list of coins for which historical data has to be downloaded.
def setupExchanges(list_of_exchanges):
    done = False
    i=0
    #df_markets = pd.DataFrame(markets)
    #bittrex_market = bittrex_exchange.fetchMarkets()
    #binance_market = binance_exchange.fetchMarkets()
    #kucoin_market = kucoin_exchange.fetchMarkets()
    #list_of_markets = [#bittrex_market,
                       #binance_market
     #                  kucoin_market #For kucoin the fetchMarkets function returns different dictionary keys
      #                  ]

    list_of_tuples = []
    for exchange in list_of_exchanges:
        coins_list = set()
        #if exchange.name == 'Cryptopia' or exchange.name == 'Bittrex' or exchange.name == 'Kucoin' or exchange.name == 'Huobi Pro':
            #continue #exchange.name == 'Binance' or 
        markets = exchange.fetchMarkets()
        pprint.pprint(markets[0])
        sys.exit(0)
        for row in markets:
            #row.base,row.quote,row.symbol,row.type,row.active
            #print(row.keys())
            tuple_data = (exchange.name,row['base'],row['quote'],row['symbol'],row['active'])
            list_of_tuples.append(tuple_data)
            
    return list_of_tuples

In [ ]:
setupExchanges(list_of_exchanges)


## For All the Exchanges, if it sends Historical Trade Data, Get It, Some Exchanges Only Send Recent Trade Data(Top 100),
## others will send complete Trade Data. Define interval - (todays time in UTC milliseconds - 1 day,todays time in UTC milliseconds)
 ## and keep going back
 

In [ ]:
def fetchTrades(exchange,symbol,start_time,end_time,how_many_result_limit,api_rate_limit,current_trade_id):
        #since = milliseconds - 86400000  # -1 day from now
        # alternatively, fetch from a certain starting datetime
        all_trades = []
        first_order_time = -1
        while start_time < end_time:
            print(start_time,end_time,len(all_trades),current_trade_id)
            trades =  exchange.fetch_trades(symbol, start_time, how_many_result_limit)
            time.sleep(api_rate_limit)
            #pprint.pprint(trades[0])
            #sys.exit(0)
            if len(trades):
                start_time = trades[len(trades) - 1]['timestamp']
                if trades[len(trades) - 1]['id'] == current_trade_id:
                    break
                current_trade_id = trades[len(trades) - 1]['id']
                if first_order_time == -1:
                    first_order_time = trades[0]['timestamp']
                all_trades += trades
            else:
                break
            #print(start_time,end_time)
        return (all_trades,first_order_time,current_trade_id)

In [ ]:
#for exchange in list_of_exchanges:
all_trades = []
for exchange in list_of_exchanges:
    if exchange.has['fetchTrades']:
        markets = exchange.fetchMarkets()
        for row in markets:
            start_time = exchange.milliseconds() - one_day_in_milliseconds
            end_time = exchange.milliseconds()
            rate_limit = exchange.rateLimit/1000
            got_trades = True
            i=1
            current_trade_id = -1
            symbol = row['symbol']
            while(got_trades):
                print(start_time,end_time,'hello',len(all_trades))
                trades_and_first_trade_time = fetchTrades(exchange,symbol,start_time,end_time,
                                                                      limit_of_trades_to_be_downloaded_from_exchange[exchange],
                                                         rate_limit,current_trade_id)
                if len(trades_and_first_trade_time[0]):
                    if trades_and_first_trade_time[2] == current_trade_id:
                        break
                    current_trade_id = trades_and_first_trade_time[2]
                    all_trades += trades_and_first_trade_time[0]
                    end_time = start_time
                    start_time = start_time-one_day_in_milliseconds
                else:
                    got_trades = False
                    break


In [ ]:
def fetchOrders(exchange,symbol,since,limit,milliseconds):
    if exchange.has['fetchOrders']:
        since = milliseconds - 86400000  # -1 day from now
        # alternatively, fetch from a certain starting datetime
        # since = exchange.parse8601('2018-01-01T00:00:00Z')
        all_orders = []
        while since < exchange.milliseconds ():
            symbol = None  # change for your symbol
            limit = 20  # change for your limit
            orders = await exchange.fetch_orders(symbol, since, limit)
            if len(orders):
                since = orders[len(orders) - 1]['timestamp']
                all_orders += orders
            else:
                break

In [ ]:
def fetchOHLCV(exchange,symbol,timeframe,start_time,end_time,how_many_result_limit,api_rate_limit):
    all_OHLCV = []
    first_OHLCV_Candle_time = -1
    previous_start_time = -1
    while start_time < end_time:
        print(start_time,end_time,len(all_OHLCV))
        OHLCV_data =  exchange.fetchOHLCV(symbol,timeframe, start_time, how_many_result_limit)
        time.sleep(api_rate_limit)
        #pprint.pprint(trades[0])
        #sys.exit(0)
        if len(OHLCV_data):
            if previous_start_time == start_time:
                break
            previous_start_time = start_time
            start_time = OHLCV_data[len(OHLCV_data) - 1][0]
            if first_OHLCV_Candle_time == -1:
                first_OHLCV_Candle_time = OHLCV_data[0][0]
            all_OHLCV += OHLCV_data
        else:
            break
        #print(start_time,end_time)
    return (all_OHLCV,first_OHLCV_Candle_time)

In [ ]:
all_OHLCV = []
for exchange in list_of_exchanges:
    if exchange.has['fetchOHLCV']:
        timeframes = exchange.timeframes.keys()
        for timeframe in timeframes:
            markets = exchange.fetchMarkets()
            for row in markets:
                start_time = exchange.milliseconds() - one_day_in_milliseconds
                end_time = exchange.milliseconds()
                rate_limit = exchange.rateLimit/1000
                gotOHLCV = True
                #current_trade_id = -1
                symbol = row['symbol']
                while(gotOHLCV):
                    print(start_time,end_time,'hello',len(all_OHLCV))
                    OHLCV_data = fetchOHLCV(exchange,symbol,timeframe,start_time,end_time,
                                                                          limit_of_trades_to_be_downloaded_from_exchange[exchange],
                                                             rate_limit)
                    if len(OHLCV_data[0]):
                        all_OHLCV += OHLCV_data[0]
                        end_time = start_time
                        start_time = start_time-one_day_in_milliseconds
                    else:
                        gotOHLCV = False
                        break
        

In [ ]:
tickers_list = []
def fetchTickers():
    for exchange in list_of_exchanges:
        if exchange.has['fetchTickers']:
            tickers = exchange.fetch_tickers()
            tickers_list.append(tickers)
        else:
            markets = exchange.fetch_markets()
            for row in markets:
                tickers = exchange.fetch_ticker(row['symbol'])
                tickers_list.append(tickers)

In [ ]:
#Experimental Cell
exchange = coinbasePro
if exchange.has['fetchTickers']:
    tickers = exchange.fetch_tickers()
    print(tickers[0])
else:
    ticker = exchange.fetch_ticker('ETH/BTC');
    print(ticker)

In [ ]:
def get_coin_metrics_data(data_parameter):
    data = function_period_mapping[data_parameter]()
    #print(data)
    return data

In [ ]:
get_coin_metrics_data('metrics')

In [ ]:
metric_info_json_file = get_coin_metrics_data('metric_info')
metric_info_json_file['metricsInfo']

In [79]:
print(pd.json_normalize(metric_info_json_file['metricsInfo']))

                    id                                               name  \
0            AdrActCnt                           Addresses, active, count   
1               BlkCnt                                       Block, count   
2          BlkSizeByte                                 Block, size, bytes   
3      BlkSizeMeanByte                           Block, size, mean, bytes   
4           CapMVRVCur               Capitalization, MVRV, current supply   
5        CapMrktCurUSD        Capitalization, market, current supply, USD   
6           CapRealUSD                      Capitalization, realized, USD   
7             DiffMean                                   Difficulty, mean   
8           FeeMeanNtv              Fees, transaction, mean, native units   
9           FeeMeanUSD                       Fees, transaction, mean, USD   
10           FeeMedNtv            Fees, transaction, median, native units   
11           FeeMedUSD                     Fees, transaction, median, USD   